In [ ]:
%pip install open_clip_torch

In [ ]:
import torch
from PIL import Image
from tkinter import filedialog
import open_clip

open_clip.list_pretrained()

In [2]:
model, _, preprocess = open_clip.create_model_and_transforms('convnext_base_w', pretrained='laion2b_s13b_b82k_augreg')
model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
tokenizer = open_clip.get_tokenizer('ViT-B-32')

In [22]:
image_path = filedialog.askopenfilename(title="Choose a image", filetypes=[("Image files", "*.png *.jpg *.jpeg")])

image = preprocess(Image.open(image_path)).unsqueeze(0)

location_labels = ["play football", "eat food", "man fighinting"]

locationn_description = tokenizer(["An image about: " + location for location in location_labels])

In [23]:
with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(locationn_description)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    location_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

# Sort the location descriptions by their probabilities in descending order
sorted_probs_and_labels = sorted(zip(location_labels, location_probs[0]), key=lambda x: x[1], reverse=True)

# Print the sorted results
for location, prob in sorted_probs_and_labels:
    print(f"{location}: {100*prob.item():.4f}%")

c:\Users\ADMIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\amp\autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


play football: 99.7516%
man fighinting: 0.2398%
eat food: 0.0086%
